<a href="https://colab.research.google.com/github/KritikaIyer17/Song_Genre_Classifier/blob/master/Text_Cleaning_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
path1 = "/content/drive/My Drive/data/lyrics1.csv"
df = pd.read_csv(path1)

In [ ]:
import numpy as np
df['ranker_genre'] = np.where((df['ranker_genre'] == 'screamo')|(df['ranker_genre'] == 'punk rock')|(df['ranker_genre'] == 'heavy metal'),'alt rock',df['ranker_genre'])
group = ['song', 'year', 'album', 'genre', 'artist', 'ranker_genre']
lyrics_by_song = df.sort_values(group).groupby(group).lyric.apply(' '.join).apply(lambda x: x.lower()).reset_index(name='lyric')
lyrics_by_song["lyric"] = lyrics_by_song['lyric'].str.replace(r'[^\w\s]','')
from sklearn.utils import shuffle
from nltk.corpus import stopwords
genres = ['Country','alt rock', 'Hip Hop','pop','rhythm and blues']
LYRIC_LEN = 400
N = 10000
RANDOM_SEED = 200
train_df = pd.DataFrame()
test_df = pd.DataFrame()

In [ ]:
for genre in genres:
    subset = lyrics_by_song[(lyrics_by_song.ranker_genre == genre) & (lyrics_by_song.lyric.str.len() >= LYRIC_LEN)]
    train_set = subset.sample(n=N, replace=True, random_state=RANDOM_SEED)
    test_set = subset.drop(train_set.index)
    train_df = train_df.append(train_set)
    test_df = test_df.append(test_set)
train_df = shuffle(train_df)
test_df = shuffle(test_df)

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
stop = list(set(stopwords.words('english')))
wnl = WordNetLemmatizer()
def tokenizer(x):
  return (wnl.lemmatize(w) for w in word_tokenize(x) if len(w)>=2 and w.isalnum())
text_clf = Pipeline([('vect', TfidfVectorizer(ngram_range=(1, 2),tokenizer=tokenizer,stop_words=stop,max_df=0.4,min_df=4)),('tfidf', TfidfTransformer()),('clf', MultinomialNB(alpha=0.1))])
text_clf.fit(train_df.lyric, train_df.ranker_genre)

In [ ]:
predicted = text_clf.predict(test_df.lyric)
np.mean(predicted == test_df.ranker_genre)

0.8338469608550259